# Analysis of [McFarland et al 2020] data

In [ ]:
import os, sys
import numpy as np
import pandas as pd
import scipy
import matplotlib.pyplot as plt
import scipy.io
import seaborn as sns
import gc
from anndata import AnnData
from joblib import Parallel, delayed
import scanpy
%config IPCompleter.use_jedi = False

figure_folder = './figures/McFarland/'

## Import data

In [ ]:
data_folder = '../data/McFarland/raw/'
processed_folder = '/../data/McFarland/processed'

### Unzip all files

In [ ]:
for x in os.listdir(data_folder):
    file = '%s/%s'%(data_folder, x)
    !unzip {file} -d {data_folder}

### Read folders

In [ ]:
def read_data(folder):
    data = scipy.io.mmread(folder + 'matrix.mtx').toarray()
    proportion_non_zeros = np.where(data > 0)[0].shape[0] / (data.shape[0] * data.shape[1])
    print('%2.1f %% of non-zeros'%(proportion_non_zeros * 100))

    genes = pd.read_csv(folder + 'genes.tsv', sep='\t', header=None)
    genes.columns = ['ensembl', 'hugo']
    assert genes.shape[0] == data.shape[0]

    data_df = pd.DataFrame(data.T, columns = genes['hugo'])
    barcode_df = pd.read_csv(folder + 'barcodes.tsv', header=None).values.flatten()
    classification_df = pd.read_csv(folder + 'classifications.csv')

    # Parse sample names
    samples = classification_df[['barcode', 'singlet_ID', 'singlet_margin', 'percent.mito', 'cell_quality', 'DepMap_ID']]
    samples['tissue'] = samples['singlet_ID'].apply(lambda x: '_'.join(str(x).split('_')[1:]))
    samples['cell_line'] = samples['singlet_ID'].apply(lambda x: str(x).split('_')[0])
    samples = samples.set_index('barcode').loc[barcode_df].reset_index()

    # Link to data    
    samples['expt'] = folder.split('/')[-2]
    data_df.index = pd.MultiIndex.from_frame(samples)
    
    return data_df

In [ ]:
expts_data_df = []

folders = [f for f in os.listdir(data_folder) if os.path.isdir('%s/%s'%(data_folder, f))]
expts_data_df = Parallel(n_jobs=30, verbose=10)(
    delayed(read_data)('%s/%s/'%(data_folder, f))
    for f in folders
)
gc.collect()

In [ ]:
data_df = expts_data_df[0]

for idx, df in enumerate(expts_data_df[1:]):
    print('MERGE %s'%(idx))
    data_df = pd.concat([data_df, df])
    expts_data_df[idx+1] = None
    gc.collect()

## NSCLC cell lines filtering

In [ ]:
ccle_annot_df = pd.read_csv('../data/cell_lines/sample_info.csv')
ccle_annot_df = ccle_annot_df[ccle_annot_df['lineage_subtype'] == 'NSCLC']

overlappinp_nsclc_cell_lines = np.intersect1d(
    data_df.index.get_level_values('DepMap_ID'),
    np.unique(ccle_annot_df['DepMap_ID'].astype(str))
).astype(str)

print('%s OVERLAPPING CELL LINES'%(overlappinp_nsclc_cell_lines.shape[0]))

nsclc_data_df = data_df.iloc[
    data_df.index.get_level_values('DepMap_ID').isin(overlappinp_nsclc_cell_lines)
]

print('FROM %s to %s cells'%(data_df.shape[0], nsclc_data_df.shape[0]))

# Create lung AnnData
nsclc_index = pd.DataFrame(nsclc_data_df.reset_index()[nsclc_data_df.index.names])
nsclc_index['barcode'] = nsclc_index['barcode'] + '_' + nsclc_index['expt']
nsclc_data_an = AnnData(
    nsclc_data_df.values,
    obs=nsclc_index.reset_index(drop=True),
    var=pd.DataFrame(nsclc_data_df.columns)
)

### QC metrics

In [ ]:
nsclc_qc_metrics = scanpy.pp.calculate_qc_metrics(nsclc_data_an)

ax = sns.jointplot(
        "log1p_total_counts", "log1p_n_genes_by_counts",
        data=nsclc_qc_metrics[0], kind="hex"
    )
ax.ax_joint.xaxis.label.set_size(20)
ax.ax_joint.xaxis.label.set_color('black')
ax.ax_joint.yaxis.label.set_size(20)
ax.ax_joint.yaxis.label.set_color('black')

plt.tight_layout()
plt.savefig('%s/NSCLC_QC_plot.png'%(figure_folder), dpi=300)

### Filter cell

In [ ]:
min_genes = 200
filter_cells = scanpy.pp.filter_cells(nsclc_data_an, 
                                      min_genes=min_genes)
print('Going from %s cells to %s cells'%(nsclc_data_df.shape[0], nsclc_data_an.shape[0]))

### Filter genes

In [ ]:
min_cells = 3
filter_genes = scanpy.pp.filter_genes(nsclc_data_an,
                                      min_cells=min_cells)
print('Going from %s genes to %s genes'%(nsclc_data_df.shape[1], nsclc_data_an.shape[1]))

### Mitochondrial genes

In [ ]:
gene_lookup_df = pd.read_csv('../data/genes/pybiomart_gene_status.csv', 
                             sep='\t', index_col=0)
gene_lookup_df = gene_lookup_df[['Hugo', 'chromosome_name', 'status']].drop_duplicates()

protein_coding_df = gene_lookup_df[gene_lookup_df['status'] == 'protein_coding']
print('%s protein coding genes from pybiomart'%(protein_coding_df.shape[0]))

chromosome = np.concatenate([np.arange(1,23).astype(str), ['X', 'Y']])
non_mitochondrial_df = gene_lookup_df[gene_lookup_df['chromosome_name'].isin(chromosome)]
mitochondrial_df = gene_lookup_df[gene_lookup_df['chromosome_name'] == 'MT']

In [ ]:
NSCLC_MT_prop_df = nsclc_data_an.to_df().T
NSCLC_MT_prop_df['IS_MT'] = (np.isin(nsclc_data_an.var['hugo'].values, mitochondrial_df['Hugo']))

NSCLC_MT_prop_df = NSCLC_MT_prop_df.groupby('IS_MT').agg('sum').T
NSCLC_MT_prop_df = (NSCLC_MT_prop_df.T / np.sum(NSCLC_MT_prop_df, axis=1)).T

plt.figure(figsize=(4,6))
sns.violinplot(y=NSCLC_MT_prop_df[True], orient='v', alpha=0.7)
# sns.swarmplot(y=NSCLC_MT_prop_df[True].values, color='black', size=2)

plt.ylabel('MT counts / all counts per cell', fontsize=20, color='black')
plt.yticks(fontsize=15, color='black')
plt.title('MT proportion', fontsize=20, color='black')
plt.tight_layout()
plt.savefig('%s/NSCLC_MT_proportion.png'%(figure_folder), dpi=300)

In [ ]:
mito_filtering_params = {
    'min': 0.02,
    'max': 0.5
}

mito_filtered_samples = NSCLC_MT_prop_df[(NSCLC_MT_prop_df[True] < mito_filtering_params['max'])\
                                         & (NSCLC_MT_prop_df[True] > mito_filtering_params['min'])].index
print('%s cells filtered'%(nsclc_data_an.shape[0] - mito_filtered_samples.shape[0]))
nsclc_data_an = nsclc_data_an[mito_filtered_samples]

### Ribosomal genes

In [ ]:
ribosomal_genes_df = pd.read_csv(
    '../data/genes/ribosomal_genes.csv', 
    sep=',', index_col=0, skiprows=1
)

ribosomal_genes = ribosomal_genes_df['Gene'].values.astype(str)

In [ ]:
NSCLC_ribo_prop_df = nsclc_data_an.to_df().T
NSCLC_ribo_prop_df['IS_RIBO'] = np.isin(nsclc_data_an.var['hugo'].values, ribosomal_genes)

NSCLC_ribo_prop_df = NSCLC_ribo_prop_df.groupby('IS_RIBO').agg('sum').T
NSCLC_ribo_prop_df = (NSCLC_ribo_prop_df.T / np.sum(NSCLC_ribo_prop_df, axis=1)).T

plt.figure(figsize=(4.5,6))
sns.violinplot(y=NSCLC_ribo_prop_df[True], orient='v', alpha=0.7)

plt.ylabel('Ribosomal counts / all counts \n (per cell)', fontsize=20, color='black')
plt.yticks(fontsize=15, color='black')
plt.title('Ribosomal gene proportion', fontsize=20, color='black')
plt.tight_layout()
plt.savefig('%s/NSCLC_Ribo_proportion.png'%(figure_folder), dpi=300)

In [ ]:
ribo_filtering_params = {
    'min': 0.1,
    'max': 0.6
}

ribosomal_filtered_samples = NSCLC_ribo_prop_df[(NSCLC_ribo_prop_df[True] < ribo_filtering_params['max'])\
                                                & (NSCLC_ribo_prop_df[True] > ribo_filtering_params['min'])].index
print('%s cells filtered'%(NSCLC_ribo_prop_df.shape[0] - ribosomal_filtered_samples.shape[0]))
nsclc_data_an = nsclc_data_an[ribosomal_filtered_samples]

### Restriction to protein coding

In [ ]:
data_pc_idx = np.isin(nsclc_data_an.var['hugo'].values, protein_coding_df['Hugo'].values)
data_pc_genes = nsclc_data_an.var[data_pc_idx]
nsclc_data_an = nsclc_data_an[:,data_pc_idx]
print('%s PC genes'%(data_pc_genes.shape[0]))

### Highly variable questions

In [ ]:
n_top_genes = 3000

scanpy.pp.highly_variable_genes(nsclc_data_an, 
                                n_top_genes=n_top_genes, 
                                flavor='seurat_v3')

high_var_genes = nsclc_data_an.var[nsclc_data_an.var['highly_variable']].sort_values('highly_variable_rank')['hugo']
high_var_genes = np.array(high_var_genes).astype(str)

In [ ]:
print('%s highly variable genes'%(high_var_genes.shape[0]))
print('%s are protein coding'%(np.intersect1d(high_var_genes, protein_coding_df['Hugo'].values).shape[0]))
print('%s are MT'%(np.intersect1d(high_var_genes, mitochondrial_df['Hugo'].values).shape[0]))
print('%s are ribosomal'%(np.intersect1d(high_var_genes, ribosomal_genes).shape[0]))

### Check and remove outliersm

In [ ]:
nsclc_data_filtered_an = nsclc_data_an[:,nsclc_data_an.var['highly_variable']]

#### Gene-level: number of cells expressing a gene

In [ ]:
plot_df = np.sum(nsclc_data_filtered_an.to_df() != 0, axis=0) / nsclc_data_filtered_an.shape[0]

# fig, axes = plt.subplots(1,2, figsize=(8,5))
axes = plt.figure(constrained_layout=True, figsize=(10,5)).subplot_mosaic(
    """
    ABBB
    """
)
sns.violinplot(y=plot_df, orient='v', ax=axes['A'])
axes['A'].set_ylim(-0.05, 1.05)
axes['A'].set_ylabel('Proportion of non zero per gene', fontsize=20, color='black')
axes['A'].tick_params(axis='both', which='major', labelsize=15)

axes['B'].plot(plot_df.sort_values().values, linewidth=3)
axes['B'].set_ylim(-0.05, 1.05)
axes['B'].tick_params(axis='both', which='major', labelsize=15)
axes['B'].set_xlabel('Gene rank', fontsize=20, color='black')

plt.tight_layout()
plt.savefig('%s/NSCLC_gene_dropout_rank.png'%(figure_folder), dpi=300, facecolor='white')
plt.show()

del plot_df

#### Sample-level library size

In [ ]:
library_size_df = np.sum(nsclc_data_an.to_df(), axis=1)

axes = plt.figure(constrained_layout=True, figsize=(10,5)).subplot_mosaic(
    """
    ABBB
    """
)
sns.violinplot(y=library_size_df, orient='v', ax=axes['A'])
axes['A'].set_ylabel('Library size per single cell', fontsize=20, color='black')
axes['A'].tick_params(axis='both', which='major', labelsize=15)

axes['B'].plot(library_size_df.sort_values().values, linewidth=3)
axes['B'].tick_params(axis='both', which='major', labelsize=15)
axes['B'].set_xlabel('Cell rank', fontsize=20, color='black')

plt.tight_layout()
plt.savefig('%s/library_size.png'%(figure_folder), dpi=300, facecolor='white')
plt.show()

In [ ]:
threshold_library_size = {'min_library_size': 500, 'max_library_size':75000}

selected_cells = (library_size_df > threshold_library_size['min_library_size']) 
selected_cells = selected_cells & (library_size_df < threshold_library_size['max_library_size'])
print('%s cells selected out of %s: %s %%'%(
    np.sum(selected_cells),
    nsclc_data_filtered_an.shape[0],
    np.sum(selected_cells) / nsclc_data_filtered_an.shape[0] * 100
))

In [ ]:
nsclc_data_filtered_an = nsclc_data_filtered_an[selected_cells]

#### Total expression

In [ ]:
gene_total_exp_df = np.sum(nsclc_data_filtered_an.to_df(), axis=0)

# fig, axes = plt.subplots(1,2, figsize=(8,5))
axes = plt.figure(constrained_layout=True, figsize=(10,5)).subplot_mosaic(
    """
    ABBB
    """
)
sns.violinplot(y=gene_total_exp_df, orient='v', ax=axes['A'])
axes['A'].set_ylabel('Proportion of non zero per gene', fontsize=20, color='black')
axes['A'].tick_params(axis='both', which='major', labelsize=15)

axes['B'].plot(gene_total_exp_df.sort_values().values, linewidth=3, marker='+')
axes['B'].tick_params(axis='both', which='major', labelsize=15)
axes['B'].set_xlabel('Gene rank', fontsize=20, color='black')

plt.tight_layout()
plt.savefig('%s/NSCLC_gene_total_exp.png'%(figure_folder), dpi=300, facecolor='white')
plt.show()

### Save
#### Protein coding

In [ ]:
nsclc_data_df[data_pc_genes['hugo'].values].to_csv(
    '../data/McFarland/processed/NSCLC_protein_coding.csv'
)
nsclc_data_df[data_pc_genes['hugo'].values].to_pickle(
    '../data/McFarland/processed/NSCLC_protein_coding.pkl',
    compression='gzip'
)

nsclc_data_df[data_pc_genes['hugo'].values].index.to_frame().reset_index(drop=True).to_csv(
    '../data/McFarland/processed/NSCLC_metadata.csv',
    sep=','
)

#### Highly variable

In [ ]:
nsclc_save_df = pd.DataFrame({
    'min_cells': [min_cells],
    'min_genes': [min_genes],
    'n_top_genes': [n_top_genes],
    'min_library_size': [threshold_library_size['min_library_size']],
    'max_library_size': [threshold_library_size['max_library_size']],
    'min_ribosomal_filtering': [ribo_filtering_params['min']],
    'max_ribosomal_filtering': [ribo_filtering_params['max']]
}).T

In [ ]:
nsclc_data_filtered_an.obs['pool'] = nsclc_data_filtered_an.obs['expt'].str.extract(r'_(expt[0-9]*)')
nsclc_data_filtered_df = nsclc_data_filtered_an.to_df()
nsclc_data_filtered_df.columns = nsclc_data_filtered_an.var['hugo']
nsclc_data_filtered_df.index = pd.MultiIndex.from_frame(nsclc_data_filtered_an.obs[['barcode', 'singlet_ID', 'pool', 'expt']])
nsclc_data_filtered_df.index.names = ['barcode', 'sample', 'pool', 'expt']

nsclc_data_filtered_df.to_csv(
    '../data/McFarland/processed/NSCLC_highly_variable.csv'
)
nsclc_data_filtered_df.to_pickle(
    '../data/McFarland/processed/NSCLC_highly_variable.pkl',
    compression='gzip'
)